In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

!ngrok config add-authtoken AUTH_TOKEN # Add auth token here


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Streamlit code

In [ ]:
%%writefile app.py
import streamlit as st
import os

st.set_page_config(page_title="Flashcard Generator")
st.title("Flashcard Generator")
input_method = st.radio("Choose input method:", ["Upload file", "Paste text manually"])

text_content = ""

if input_method == "Upload file":
    uploaded_file = st.file_uploader("Upload a .pdf or .txt file", type=["pdf", "txt"])
    if uploaded_file is not None:
        if uploaded_file.name.endswith(".txt"):
            text_content = uploaded_file.read().decode("utf-8")
        elif uploaded_file.name.endswith(".pdf"):
            import pdfplumber
            with pdfplumber.open(uploaded_file) as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text() + "\n"
                text_content = text
else:
    text_content = st.text_area("Paste text here:", height=250)

# Number of flashcards
num_flashcards = st.number_input("Number of flashcards", min_value=1, max_value=10, value=5)

# Submit button
if st.button("🚀Generate Flashcards"):
    with open("input.txt", "w", encoding="utf-8") as f:
        f.write(text_content)
    with open("num.txt", "w") as f:
        f.write(str(num_flashcards))
    st.success("Model will now generate flashcards. Please wait...")

# Refresh
if st.button("🔄 Refresh Output"):
    if os.path.exists("output.txt"):
        with open("output.txt", "r", encoding="utf-8") as f:
            flashcards = f.read()
        st.subheader("🧠 Generated Flashcards:")
        st.text_area("Flashcards Output", flashcards, height=400)
    else:
        st.error("❌ No flashcards found. Please run the model cell in Colab first.")


Writing app.py


In [7]:
!pip install -q transformers accelerate bitsandbytes pdfplumber pandas huggingface_hub


Pyngrok

In [30]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("\n🌐 Public URL to use Streamlit app:", public_url)
!streamlit run app.py &>/content/logs.txt &


🌐 Public URL to use Streamlit app: NgrokTunnel: "https://cf08-34-125-109-124.ngrok-free.app" -> "http://localhost:8501"


Model code

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import notebook_login


# Authenticate with Hugging Face Hub
notebook_login()

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Run this after providing input in Streamlit

In [31]:
with open("input.txt", "r", encoding="utf-8") as f:
    text_content = f.read()
with open("num.txt", "r") as f:
    num_flashcards = int(f.read())


prompt = f"""
You are an AI assistant generating educational flashcards.
Generate exactly {num_flashcards} flashcards from the following text.
Each flashcard format:
Q: <question here>
A: <answer here>
Text: \"\"\"{text_content}\"\"\"
"""

inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=700)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


# Save to output file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(output_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
